In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import re

import os
import glob
import pdb

import matplotlib
from matplotlib import rc
from matplotlib import pyplot as plt
from cycler import cycler

from collections.abc import Mapping, Sequence

import json
import yaml


from dataclasses import dataclass, asdict

import tqdm

import IPython
from IPython.display import display

from io import StringIO

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

# hueristic way to navigate one folder above the notebooks directory
if 'notebooks' in os.getcwd():
    os.chdir('..')
    
! pwd
print(1)    
#from train import Config
print(2)
#from vtab import VTAB_DATASET_NAMES, ADDITIONAL_DATASET_NAMES, DATASET_NAMES, SPECIALIZED_DATASET_NAMES, SPECIALIZED_DATASET_NAMES
print(3)

# host = !echo $HOST
# if 'c-0' in host[0]:
#     os.chdir(os.path.expanduser('~/expriment-starter'))
# else:
#     os.chdir('/Users/ycarmon/Drive/Research/DoG/vision_experiments')
    
# from src import Config

# general utils

In [ ]:
def modified_groupby(df, labels):
    if type(labels) == str:
        df.groupby(labels, dropna=False)

    if len(labels) == 0:
        return [("", df)]
    
    if len(labels) == 1:
        labels = labels[0]
    
    return df.groupby(labels, dropna=False)

In [ ]:
def read_experiments(experiment_names, index_field='epoch', read_train=False, folder='results'):
    experiment_names = [os.path.join(folder, ename) for ename in experiment_names]
    paths_to_plot = [os.path.join(ename, fname) for ename in experiment_names 
                     for fname in os.listdir(ename) if not (fname.endswith('.yaml') or fname.endswith('.sh') or 'logs' in fname)] 
    short_names = paths_to_plot #[n.replace(experiment_name + '_', '') for n in folders_to_plot]
    # short_names

    _RE_COMBINE_WHITESPACE = re.compile(r"\s+")

    train_dfs = {}
    eval_dfs = {}
    configs = {}
    for folder_name, short_name in tqdm.tqdm(list(zip(paths_to_plot, short_names))):
        is_done = os.path.exists(os.path.join(folder_name, 'done'))

        eval_path = os.path.join(folder_name, 'stats_eval.csv')
        eval_path_tmp = os.path.join(folder_name, 'stats_eval_tmp.csv')
        with open(eval_path, 'r') as f:
            csv_text = f.read()
        if csv_text[0] != ',':
            print(short_name)
            continue
        csv_text = csv_text.replace(',[',',"[')
        csv_text = csv_text.replace('],',']",')
        csv_text = csv_text.replace('[ ','[')
        csv_text = csv_text.replace(' ]',']')
        csv_text = csv_text.replace('. ','.0')
        csv_text = ' '.join(csv_text.split(sep=' '))
        with open(eval_path_tmp, 'w') as f:
            f.write(csv_text)
        try:
            df = pd.read_csv(eval_path_tmp)#(StringIO(csv_text))#(eval_path)
        except:
            continue
        
        with open(os.path.join(folder_name, 'args.yaml'), 'r') as f:
            configs[short_name] = yaml.safe_load(f) # asdict(Config(**yaml.safe_load(f)))
    #     df['name'] = short_name
        configs[short_name]['is_done'] = is_done
        if not 'loss' in configs[short_name]:
            configs[short_name]['loss'] = 'none'

        
        df = df.rename(columns={'Unnamed: 0': 'epoch'}).rename(columns=lambda x: x.replace('eval/', 'test/'))
        #print(short_name, len(df.columns), df.columns)
        df['epoch'] += 1

        assert(index_field != 'step')
        #df['step'] = df['train/seen_examples'] / configs[short_name]['batch_size']
        
        if read_train:
            assert(False)
            train_df = pd.read_csv(os.path.join(folder_name, 'stats_train.csv'))
            train_df = train_df.rename(columns={'Unnamed: 0': 'step'})
            train_df['step'] = (train_df['step'] + 1) / configs[short_name]['grad_accum_steps']
            train_df['epoch'] = ((train_df['seen_examples'] - 1) / df.iloc[0]['train/seen_examples']).astype(int)
            train_df = train_df[ train_df['seen_examples'] % configs[short_name]['batch_size'] == 0 ]
            train_df = train_df.set_index(index_field)
            train_dfs[short_name] = train_df
        
        if os.path.exists(os.path.join(folder_name, 'stats_regret.csv')):
            regret_df = pd.read_csv(os.path.join(folder_name, 'stats_regret.csv')).set_index('epoch')
            regret_df['regret_ratio'] = regret_df.regret / regret_df.loss_diff
            regret_df = pd.concat([split_df.rename(columns=lambda x: split + '/' + x) for split, split_df in modified_groupby(regret_df,'split')], axis=1)
            df = pd.concat([df.set_index('epoch'), regret_df], axis=1).reset_index()
        
        df = df.set_index(index_field)
        eval_dfs[short_name] = df
        # eval_dfs[short_name] = eval_dfs[short_name].rename(columns=lambda n: n.replace('accuracy_av', 'accuracy/average'))
    #     train_dfs[short_name] = modified_groupby(pd.read_csv(os.path.join(experiment_name, folder_name, 'stats_train.csv')),'epoch').mean()
        

    print('Successfully read %d out of %d folders' % (len(eval_dfs) ,len(paths_to_plot)))

    keys = set([c for df in eval_dfs.values() for c in df.columns])  # remove 

    # turn every run into a "row" in a dataframe that also contains the entire config
    big_df = pd.DataFrame(configs).T.rename_axis('name').sort_index()
    for key in keys:
        big_df[key] = [eval_dfs[name][key] if key in eval_dfs[name].columns else np.array([]) for name in big_df.index]
        
    if read_train:
        keys = set([c for df in train_dfs.values() for c in df.columns])
        for key in keys:
            big_df['training/' + key] = [train_dfs[name][key] if key in train_dfs[name].columns else np.array([]) for name in big_df.index]

#     # throw away stuff that didn't finish running
# #     valid_mask = big_df.epochs == big_df['test/robust_loss'].apply(len)
#     valid_mask = big_df['eval/loss'].apply(len) > 0 # == big_df['eval_num']
#     big_df = big_df[valid_mask]
#     config_df = pd.DataFrame(configs).T.rename_axis('name').sort_index()[valid_mask]
# #     config_df['ada_eps'].fillna(1e-8, inplace=True)

    print('%d results left after filtering' % (len(big_df),))
    
#     big_df.loc[big_df['training_examples'] < 0, 'training_examples'] = np.inf

    return big_df

@dataclass
class Interval:
    start: float = -np.inf
    end: float = np.inf

def filt(df, **kwargs):
    filt_df = df.copy()
    for k, v in kwargs.items():
        if callable(v):
            filt_df = filt_df.loc[filt_df[k].apply(v)]
        elif isinstance(v, str):
            if v.startswith('~'):
                filt_df = filt_df.loc[~filt_df[k].str.contains(v[1:])]
            else:
                filt_df = filt_df.loc[filt_df[k].str.contains(v)]
        elif isinstance(v, tuple):
            for vv in v:
                filt_df = filt(filt_df, **{k: vv})
        elif isinstance(v, list):
            filt_df = pd.concat([filt(filt_df, **{k: vv}) for vv in v], axis=0)
        elif isinstance(v, float) or isinstance(v, int) or isinstance(v, bool):
            filt_df = filt_df.loc[filt_df[k]==v]
        elif isinstance(v, Interval):
            filt_df = filt_df[(v.start <= filt_df[k]) & (filt_df[k] <= v.end)]
        else:
          raise ValueError(f'Don''t know how to handle filter type {type(v)}')
    return filt_df

def pick_best(df, index_columns, target_column, k_best=1, horizon=np.inf): 
    df = df.copy()
    
    if 'loss' in target_column:
        op = lambda x: -x.truncate(after=horizon).min()
    else:
        op = lambda x: x.truncate(after=horizon).max()

    df['target'] = df[target_column].apply(op)

    if len(index_columns)  > 0:
        df = df.reset_index().set_index(index_columns).sort_index()
        best_rows = pd.DataFrame()
        for i in df.index.unique():
            best_rows = best_rows.append(df.loc[[i]].nlargest(k_best, 'target'))
    else:
        best_rows = df.nlargest(k_best, 'target')
    
#     pdb.set_trace()
    return best_rows.reset_index().set_index('name')

def nanfloat(x):
    try:
        return float(x)
    except:
        return np.nan
    
def nice_string(s):
    return s.replace('_', ' ').capitalize()

def short_string(s, strlen=12):
    return '_'.join([ss[:strlen] for ss in s.split('_')])

def config_str(ks, vs, key_len=12, sep=', '):
    if not isinstance(vs, tuple) or isinstance(vs, list):
        vs = [vs]
    return sep.join([f'{short_string(k, strlen=key_len)} = {v}' for k, v in zip(ks, vs)])

MARKERS = ['o', 'v', 'p', 'X', '<', 's', '^','P', '*', '>', 'd']
LINESTYLES = ['-', '--', ':', '-.']
COLORS = [c for c in matplotlib.colors.TABLEAU_COLORS]#[:-2]

def get_marker(i):
    return MARKERS[i % len(MARKERS)]

def get_color(i, clist=None):
    if clist is None:
        clist = COLORS
    return clist[i % len(clist)]

def get_linestyle(i):
    return LINESTYLES[i % len(LINESTYLES)]

In [ ]:
folders = np.sort(os.listdir('results'))
folders_quotes = [f"'{f}'" for f in folders]
print(f'[{", ".join(folders_quotes)}]')

# DoG

In [ ]:
def clean_nanmin(arr):
    if len(arr) == 0 or np.all(np.isnan(arr)):
        return np.nan
    return np.nanmin(arr)

def clean_nanmax(arr):
    if len(arr) == 0 or np.all(np.isnan(arr)):
        return np.nan
    return np.nanmax(arr)

In [ ]:
def init_dist_to_rbar(dist):
    if isinstance(dist, float):
        return dist
    elif isinstance(dist, str):
        dist_arr = np.fromstring(dist[1:-1], dtype=np.float64, sep=' ')
        return dist_arr#np.linalg.norm(dist_arr)
    raise  Exception('dist of unknown type ' + str(type(dist)))

def generate_dog_data_partial(df):
    accele_df = df.query('optim_alg != "dog" and optim_alg != "dog_public"').copy()
    accele_df.loc[:,'train/effective_lr'] = accele_df.loc[:,'train/alpha'] * accele_df.loc[:,'train/eta']

    return accele_df

def generate_dog_data(df, with_effective_lr = True, dog=True, adog=True):
    dog_df = df.query('optim_alg == "dog" or optim_alg == "dog_public"').copy()
    accele_df = df.query('optim_alg != "dog" and optim_alg != "dog_public"').copy()

    if dog:
        dog_df.loc[:,'train/rbar'] = dog_df.loc[:,'train/dog/init_dist'].map( lambda x: x.map( lambda y: init_dist_to_rbar(y)))
        dog_df.loc[:,'train/alpha'] = dog_df.loc[:,'train/dog/init_dist'].map( lambda x: x.map( lambda y: 1.))
        dog_df.loc[:,'train/eta'] = dog_df.loc[:,'train/dog/eta']

    if adog:
        accele_df.loc[:,'train/rbar'] = accele_df.loc[:,'train/rbar'].map( lambda x: x.map( lambda y: init_dist_to_rbar(y)))

    if dog and adog:
        new_df = pd.concat([dog_df, accele_df])
    elif dog:
        new_df = dog_df
    else:
        new_df = accele_df

    if with_effective_lr:
        new_df.loc[:,'train/effective_lr'] = new_df.loc[:,'train/alpha'] * new_df.loc[:,'train/eta']

    return new_df

def get_best_losses(*dfs):
    best_losses = {}

    for df in dfs:
        for loss in df['loss'].unique():
            best_loss = clean_nanmin( df.query(f'loss == "{loss}"').filter(regex='.*/loss').applymap( clean_nanmin ).values )
            best_losses[loss] = clean_nanmin( [best_loss, best_losses.get(loss, np.inf)] )

    return best_losses

def generate_subopt_helper(df, best_loss):
    best_loss = clean_nanmin( df.filter(regex='.*/loss').applymap( clean_nanmin ).values )
    for key in df.filter(regex='.*/loss').columns:
        subopt_key = key[:-4] + "empirical_subopt"
        df.loc[:,subopt_key] = df.loc[:,key] - best_loss

    return df

def generate_subopt(df, best_losses: dict = None):

    if best_losses is None:
        best_losses = get_best_losses(df)

    new_dfs = []
    for loss in df['loss'].unique():
        new_df = df.query(f'loss == "{loss}"').copy()
        new_df = generate_subopt_helper(new_df, best_losses[loss])
        new_dfs.append(new_df)


    new_df = pd.concat(new_dfs)
    return new_df

In [ ]:
def sgd_find_best(df, field='train/empirical_subopt', method = "best"):
    row_fields = ['model', 'dataset', 'batch_size', 'loss', 'momentum', 'activation']
    legend_fields = ['base_lr']

    best_func = clean_nanmin
    best_value_init = np.inf
    if 'accuracy' in field:
        best_func = clean_nanmax
        best_value_init = -np.inf

    arr_best = lambda arr: arr.iloc[-1]
    if method == "best":
        arr_best = lambda arr: best_func(arr)

    rows = []
    count=0
    #df['is_best'] = df['dog_granularity'].map( lambda y: False)
    for i_row, (row_vals, row_df) in enumerate(modified_groupby(df, row_fields)):

        best_value = best_value_init
        # Finding the best value
        for i_entry, (legend_vals, entry_df) in enumerate(modified_groupby(row_df,legend_fields)):
            assert(len(entry_df[field]) == 1)
            current_best_value = arr_best(entry_df[field][0])
            
            if not np.isnan(current_best_value):
                if best_value is None:
                    best_value = current_best_value
                best_value = best_func([best_value, current_best_value])

        # Add correct entry
        found = False
        for i_entry, (legend_vals, entry_df) in enumerate(modified_groupby(row_df,legend_fields)):
            last_epoch = len(entry_df[field][0])
            current_best_value = arr_best(entry_df[field][0])
            if (best_value == current_best_value):
                rows.append(entry_df)

    return pd.concat(rows)#.query('is_best')

In [ ]:
def limit_series(x, limit):
    if limit is None or limit <= 0 or isinstance(x, (np.floating,float)):
        return x
    return x[:limit] if isinstance(x, np.ndarray) else x.truncate(after=limit)

def create_last_and_best(df, limit=None):
    for keyword in ['loss', 'empirical_subopt', 'accuracy']:
        for key in df.filter(regex='.*/'+keyword).columns:

            is_float_or_empty = lambda x: isinstance(x, (np.floating,float)) or len(x) == 0

            # Last
            key_last = key[:-len(keyword)] + 'last_' + keyword
            df.loc[:,key_last] = df.loc[:,key].map( lambda x: np.nan if is_float_or_empty(limit_series(x, limit)) else limit_series(x, limit).iloc[-1] )

            # Best
            best_func = clean_nanmin
            if keyword == 'accuracy':
                best_func = clean_nanmax
            key_best = key[:-len(keyword)] + 'best_' + keyword
            df.loc[:,key_best] = df.loc[:,key].map( lambda x: np.nan if is_float_or_empty(limit_series(x, limit)) else best_func(limit_series(x, limit)) )

    return df

In [ ]:
def select_av(df, av='constant_8'):
    df = df

    for col in df.filter(regex='.*/'+ av + '/.*').columns:
        av_col = col.replace('/'+ av + '/', '/av/')
        df[av_col] = df[col]

    return df

# Table  Utils

In [ ]:
def get_best_value(df, fields, y_field, use_max=True):
    best_func = clean_nanmax if use_max else clean_nanmin
    worst_func = clean_nanmin if use_max else clean_nanmax

    best_dict = {}
    initial_dict = {}
    for i_row, (row_vals, row_df) in enumerate(modified_groupby(df,fields)):
        key =str(row_vals)
        best_dict[key] = best_func( row_df.loc[:,y_field].apply( best_func ).values )
        initial_dict[key] = worst_func( row_df.loc[:,y_field].apply( lambda x: x.iloc[0] ).values )

    return best_dict, initial_dict

def get_number_of_steps_until(df, fields, col_fields, y_field, best_dict, until, use_max=True, initial_dict=None):
    
    dfs_arr = []
    dfs_dict = {}
    key_to_vals = {}
    for i_row, (row_vals, row_df) in enumerate(modified_groupby(df,fields)):
        key = str(row_vals)
        key_to_vals[key] = row_vals
        if initial_dict is None:
            until_value = best_dict[key] * until
        else:
            assert use_max
            until_value = (best_dict[key] - initial_dict[key]) * until + initial_dict[key]
        df_copy = row_df.copy()

        from_where = df_copy.loc[:,y_field].apply( lambda x: x.apply( lambda y: (-np.inf if use_max else np.inf) if np.isnan(y) else y ) )
        from_where = from_where.apply( lambda x: x>= until_value if use_max else x<= until_value )
        from_where = from_where.apply( lambda x: x.idxmax() if x.max() else np.inf, axis='columns' )
        df_copy.loc[:,'steps_until'] = from_where
        dfs_arr.append(df_copy)

        tmp_dict = {}
        for n in df_copy.index:
            tmp_dict[n] = str(df_copy.loc[n,col_fields].values)

        dfs_dict[key] = from_where.rename( tmp_dict )
    
    new_table = pd.DataFrame.from_dict(dfs_dict, 'index')
    for key in key_to_vals:
        new_table.loc[key, fields] = key_to_vals[key]
        #for feild, val in zip(fields, key_to_vals[key]):
        #    new_table.loc[key, feild] = val

    new_table = new_table.set_index(fields)

    return new_table # pd.concat( dfs_arr ),

In [ ]:
def get_best_accuracy_val(sr, limit=None):
    last_is_better = sr['val/last/accuracy'] > sr['val/av/accuracy']
    best_val_acc = sr['val/last/accuracy'].where(last_is_better, sr['val/av/accuracy'])
    best_test_acc = sr['test/last/accuracy'].where(last_is_better, sr['test/av/accuracy'])
    if limit:
        best_val_acc[:limit]
    idx = best_val_acc.idxmax()

    return best_test_acc[idx]

def get_best_accuracy(sr, limit=None):
    last_is_better = sr['test/last/accuracy'] > sr['test/av/accuracy']
    best_test_acc = sr['test/last/accuracy'].where(last_is_better, sr['test/av/accuracy'])

    if limit:
        return best_test_acc[:limit].max()
    return best_test_acc.max()

In [ ]:
def get_best_loss_val(sr, limit=None):
    last_is_better = sr['val/last/loss'] > sr['val/av/loss']
    best_val_acc = sr['val/last/loss'].where(last_is_better, sr['val/av/loss'])
    best_test_acc = sr['test/last/loss'].where(last_is_better, sr['test/av/loss'])
    if limit:
        best_val_acc[:limit]
    idx = best_val_acc.idxmin()

    return best_test_acc[idx]

def get_best_test_loss(sr, limit=None):
    last_is_better = sr['test/last/loss'] > sr['test/av/loss']
    best_test_acc = sr['test/last/loss'].where(last_is_better, sr['test/av/loss'])

    if limit:
        return best_test_acc[:limit].min()
    return best_test_acc.min()


def get_best_train_loss(sr, limit=None):
    best_test_acc = sr['train/loss']

    if limit:
        return best_test_acc[:limit].min()
    return best_test_acc.min()

In [ ]:
def get_test_loss_tables(df, rows='optim_alg', val=False, limit=None):
    df = df.copy()
    if val:
        df['test/best/loss'] = df.apply(get_best_loss_val, args=(limit,), axis=1)
    else:
        df['test/best/loss'] = df.apply(get_best_test_loss, args=(limit,), axis=1)

    models_names = []
    for i_row, (row_vals, row_df) in enumerate(modified_groupby(df,['model'])):
        models_names.append(row_vals)

    tables = {}
    for model_name in models_names:
        model_df = df.query(f'model == "{model_name}"')

        new_df = {}
        for i_row, (row_vals, row_df) in enumerate(modified_groupby(model_df,rows)):
            new_df[row_vals] = row_df['test/best/loss'].copy()
            new_df[row_vals] = new_df[row_vals].rename(row_df['wd'])
        tables[model_name] = pd.DataFrame.from_dict(new_df, 'index')
    return tables

In [ ]:
def get_train_loss_tables(df, rows='optim_alg', limit=None):
    df = df.copy()
    df['train/best/loss'] = df.apply(get_best_train_loss, args=(limit,), axis=1)

    models_names = []
    for i_row, (row_vals, row_df) in enumerate(modified_groupby(df,['model'])):
        models_names.append(row_vals)

    tables = {}
    for model_name in models_names:
        model_df = df.query(f'model == "{model_name}"')

        new_df = {}
        for i_row, (row_vals, row_df) in enumerate(modified_groupby(model_df,rows)):
            new_df[row_vals] = row_df['train/best/loss'].copy()
            new_df[row_vals] = new_df[row_vals].rename(row_df['wd'])
        tables[model_name] = pd.DataFrame.from_dict(new_df, 'index')
    return tables

# UniXGrad

In [ ]:
markers = {'unixgrad': '+', 'accele_dog': '*', 'uni_dog': 's', 'dog_public': 'o', 'accelegrad': 'p'} 
colors = {'unixgrad': 'blue', 'accele_dog': 'orange', 'uni_dog': 'purple', 'dog_public': 'red', 'accelegrad': 'tab:purple'}
colors_bounded = {'unixgrad': {"True": 'blue', "False": 'cyan'}, 'accelegrad': {"True": 'tab:purple', "False": 'tab:pink'}}
nice_names = {'unixgrad': 'UniXGrad', 'accele_dog': 'A-DoG', 'uni_dog': 'U-DoG', 'dog_public': 'DoG', 'accelegrad': 'AcceleGrad'}
names_order = ['accele_dog', 'uni_dog', 'dog_public', 'unixgrad', 'accelegrad']
marker_size=7

import matplotlib.colors as mcolors

cmap = plt.cm.viridis
def transform(val):
    return -np.log10(1 - val)
norm = mcolors.Normalize(vmin=transform(0), vmax=transform(0.9999))

def make_sensitivity_plots(big_df_filt, epoch_selection = 'last', max_epoch_vals = [100, 400, 1000, 4000, 10000, 20000], 
                           rowsplit_keys = ['batch_size'], x_axis = 'diameter', y_axis = 'test/constant_8/accuracy', baseline_alg_mask = f'optim_alg in ("unixgrad", "accelegrad")',
                           baseline_alg_split_keys = ['optim_alg', 'bounded'], auto_y_lim_range = 0.05, y_lim = [0.8, 0.95],
                           proposed_alg_mask = f'optim_alg.str.contains("dog")', proposed_alg_split_keys = ['optim_alg'], fig=None, axes=None, colorbar=False):
    pretty_mode = False
    x_axis_categorical = False
  
    nrows = 1
    ncols = len(max_epoch_vals)
    assert not ((fig is None) ^ (axes is None))
    create_legend = fig is None
    
    for rowsplit_vals, row_df in big_df_filt.groupby(rowsplit_keys):
        hlines = []
        print(rowsplit_vals, fig is None)

        baseline_alg_df = row_df.query(baseline_alg_mask)
        proposed_alg_df = row_df.query(proposed_alg_mask)
        
        if len(baseline_alg_df) == 0:
            continue
    
        base_size = (7, 7) if not pretty_mode else (4, 4)
        if fig is None:
            print("??")
            fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(base_size[0] * ncols, base_size[1] * nrows), facecolor='w', sharey='row')
        if nrows * ncols == 1:
            axes = np.array(axes)
        axes = axes.reshape(nrows, ncols)
    
        for j, max_epoch in enumerate(max_epoch_vals):
            plt.sca(axes[0, j])
            axes[0, j].clear()
            
            if len(baseline_alg_df[y_axis].iloc[0].truncate(after=max_epoch)) == 0:
                continue
                
            baselines_to_plot = {}
            
            for i, (baseline_split_vals, baseline_split_df) in enumerate(baseline_alg_df.groupby(baseline_alg_split_keys, dropna=False)):
                if epoch_selection == 'best':
                    baseline_split_df['selected_y'] = baseline_split_df[y_axis].apply(lambda x: x.truncate(after=max_epoch).max())
                elif epoch_selection == 'last':
                    baseline_split_df['selected_y'] = baseline_split_df[y_axis].apply(lambda x: x.truncate(after=max_epoch).iloc[-1])
                else:
                    raise ValueError('?')
                to_plot = baseline_split_df.groupby(x_axis, dropna=False)['selected_y'].max()
                if x_axis_categorical:
                    # to_plot.index = to_plot.index.fillna('none')
                    to_plot.index = pd.Categorical(to_plot.index.values) #, categories=sorted(baseline_alg_df[x_axis].unique()))
                # pdb.set_trace()
            
                baselines_to_plot[config_str(baseline_alg_split_keys, baseline_split_vals)] = to_plot
                
            baselines_to_plot = pd.DataFrame(baselines_to_plot)
            baselines_to_plot.index = baselines_to_plot.index.fillna('none')
            # styles = [dict(color=get_color(i), marker=get_marker(i)) for i in range(len(baselines_to_plot))]
            # styles = [get_color(i) for i in range(len(baselines_to_plot))]
            h = baselines_to_plot.plot(ax=plt.gca(), legend=False, ms=marker_size)
            for i, ln in enumerate(h.lines):
                ln.set_marker(get_marker(i))
                # ln.set_color(get_color(i))
                #momentum = float(baselines_to_plot.columns[i].split('momentum = ')[-1])
                # pdb.set_trace()
                #ln.set_color(get_momentum_color(momentum))
                opt_alg = baselines_to_plot.columns[i].split(', ')[0].split('optim_alg = ')[-1]
                bounded = baselines_to_plot.columns[i].split(', ')[1].split('bounded = ')[-1]
                ln.set_color(colors_bounded[opt_alg][bounded])
    
            for ii, (proposed_split_vals, proposed_split_df) in enumerate(proposed_alg_df.groupby(proposed_alg_split_keys, dropna=False)):
                if epoch_selection == 'best':
                    proposed_split_df['selected_y'] = proposed_split_df[y_axis].apply(lambda x: x.truncate(after=max_epoch).max())
                elif epoch_selection == 'last':
                    proposed_split_df['selected_y'] = proposed_split_df[y_axis].apply(lambda x: x.truncate(after=max_epoch).iloc[-1])
                else:
                    raise ValueError('?')
                to_plot = proposed_split_df['selected_y'].max()

                # pdb.set_trace()
                if len(proposed_split_vals) > 1 or proposed_split_vals[0] not in colors:
                    color = colors.get(baseline_split_vals, get_color(-1-ii))
                    plt.axhline(to_plot, color=color, ls='--', lw=2, label=f'{config_str(proposed_alg_split_keys, proposed_split_vals)}')
                else:
                    color = colors[proposed_split_vals[0]]
                    marker = markers[proposed_split_vals[0]]
                    hlines.append((color, marker, to_plot))
                    plt.axhline(to_plot, color=color, lw=2, label=f'{config_str(proposed_alg_split_keys, proposed_split_vals)}')

    
            plt.title(f'Max {baseline_alg_df[y_axis].iloc[0].index.name} = {max_epoch}')
            if not x_axis_categorical:
                plt.xscale('log')
            if y_lim == 'auto':
                # y_max = min(1.0, pd.concat([baseline_alg_df, proposed_alg_df], axis=0)[y_axis].apply(lambda x: x.max()).max() + auto_y_lim_range)
                # y_min = min(0.0, pd.concat([baseline_alg_df, proposed_alg_df], axis=0)[y_axis].apply(lambda x: x.min()).min() - auto_y_lim_range)
                y_max = min(1.0, np.nanpercentile(pd.concat([baseline_alg_df, proposed_alg_df], axis=0)[y_axis].apply(lambda x: x.max()), 95) + auto_y_lim_range)
                y_min = max(0.0, np.nanpercentile(pd.concat([baseline_alg_df, proposed_alg_df], axis=0)[y_axis].apply(lambda x: x.min()), 5) - auto_y_lim_range)
                #plt.ylim(y_min, y_max)
            else:    
                plt.ylim(y_lim)
            if 'loss' in y_axis or 'subopt' in y_axis or 'rbar' in y_axis:
                plt.yscale('log')
            plt.ylabel(y_axis)
            plt.xlabel(x_axis)
    
            plt.grid(True, which='major', axis='y')
            if not x_axis_categorical:
                plt.grid(True, which='major', axis='x')
            plt.grid(True, which='minor', axis='y', color=[0.9, 0.9, 0.9], linestyle='--')
    
    
        handles, labels = plt.gca().get_legend_handles_labels()
    
        if create_legend:
            fig.legend(
                    prop={"size": 14},
            #         columnspacing=0.0,
                    ncol=5,
                    handles=handles,
                    labels=labels,
                    loc='upper center',
                    bbox_to_anchor=[0.5, -0.05],
                    frameon=True,
                    markerscale=1.0,
                )
            dataset_name = big_df_filt.iloc[0].dataset.replace('/', '+')
            model_name = big_df_filt.iloc[0].model.replace('/', '+')
            
            fig.suptitle(f'{dataset_name} {config_str(rowsplit_keys, rowsplit_vals)}')
        
            plt.tight_layout()
            plt.show()
            filename = f"figs/sensitivity/{dataset_name}-{config_str(rowsplit_keys, rowsplit_vals)} {model_name} sensitivity.pdf"
            #fig.savefig(filename, dpi=300, bbox_inches='tight')
            #fig.savefig(filename.replace('pdf', 'png'), dpi=300, bbox_inches='tight')
        else:
            metric = y_axis.replace('constant_8/', '').replace('/', ' ')
            plt.gca().set_title(f'{metric.capitalize()} after {max_epoch_vals[0]} batches')
            plt.xlabel('learning rate')

            if colorbar:
                # Create a color bar with the transformed scale
                sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
                sm.set_array([])
                cbar = plt.colorbar(sm, ax=plt.gca())
                # Set the color bar labels to the original real values
                cbar.set_ticks(transformed_momentum_values)
                cbar.set_ticklabels([f'{val:.4f}' for val in momentum_values])
                cbar.set_label('Momentum')

            x_min, x_max = plt.gca().get_xlim()
            # Plot a single marker at the rightmost edge
            for color, marker, to_plot in hlines:
                plt.gca().plot(10**-5, to_plot, marker=marker, color=color, markersize=marker_size)

In [ ]:
dog_df = read_experiments(['/a/home/cc/students/cs/kreisler/dog/vision_experiments/paper_results/24-04-04-convex_clevr_distance',
                        '/a/home/cc/students/cs/kreisler/dog/vision_experiments/paper_results/24-04-04-convex_dmlab',
                        '/a/home/cc/students/cs/kreisler/dog/vision_experiments/paper_results/24-04-04-convex_pendigits',
                        '/a/home/cc/students/cs/kreisler/dog/vision_experiments/paper_results/24-04-04-convex_resisc45',
                        '/a/home/cc/students/cs/kreisler/dog/vision_experiments/paper_results/24-04-04-convex_sun397',
                        '/a/home/cc/students/cs/kreisler/dog/vision_experiments/paper_results/24-04-04-convex_ls_svhn_vit',
                        '/a/home/cc/students/cs/kreisler/dog/vision_experiments/paper_results/24-04-04-imagenet-vit_b32-20-40',

                        '/a/home/cc/students/cs/kreisler/dog/vision_experiments/paper_results/24-04-15-convex_clevr_distance',
                        '/a/home/cc/students/cs/kreisler/dog/vision_experiments/paper_results/24-04-15-convex_dmlab',
                        '/a/home/cc/students/cs/kreisler/dog/vision_experiments/paper_results/24-04-15-convex_pendigits',
                        '/a/home/cc/students/cs/kreisler/dog/vision_experiments/paper_results/24-04-15-convex_resisc45',
                        '/a/home/cc/students/cs/kreisler/dog/vision_experiments/paper_results/24-04-15-convex_sun397',
                        '/a/home/cc/students/cs/kreisler/dog/vision_experiments/paper_results/24-04-15-convex_ls_svhn_vit',
                        '/a/home/cc/students/cs/kreisler/dog/vision_experiments/paper_results/24-04-15-imagenet-vit_b32-0.2-10',

                        '/specific/netapp5/joberant/home/maorivgi/code/dog/vision_experiments/paper_results/24-01-22-convex_clevr_distance',
                        '/specific/netapp5/joberant/home/maorivgi/code/dog/vision_experiments/paper_results/24-01-22-convex_dmlab',
                        '/home/ycarmon/no_backup/users/ycarmon/adog/paper_results/24-01-23-convex_pendigits', 
                        '/home/ycarmon/no_backup/users/ycarmon/adog/paper_results/24-01-23-convex_pendigits_more_bss',  
                        '/specific/netapp5/joberant/home/maorivgi/code/dog/vision_experiments/paper_results/24-01-21-convex_resisc45',
                        '/specific/netapp5/joberant/home/maorivgi/code/dog/vision_experiments/paper_results/24-01-21-convex_sun397',
                        '/home/ycarmon/no_backup/users/kreisler/accelerated_dog/paper_results/24-03-26-convex_ls_svhn_vit',
                        '/home/ycarmon/no_backup/users/ycarmon/adog/paper_results/24-01-24-imagenet-vit_b32'
      #'23-10-18-vtab-accelerate-dog-test'
                                #'23-09-13-linear_cifar100_resnet50', '23-09-13-linear_cifar100_vit_base_patch32_224_in21k',
                                #'23-09-13-linear_clevr_distance_resnet50', '23-09-13-linear_clevr_distance_vit_base_patch32_224_in21k',
                                #'23-09-13-linear_dsprites_ori_resnet50', '23-09-13-linear_dsprites_ori_vit_base_patch32_224_in21k',
                                #'23-09-13-linear_svhn_resnet50', '23-09-13-linear_svhn_vit_base_patch32_224_in21k'
                          ])#, index_field='step')
dog_df = dog_df.query('optim_alg == "dog" or optim_alg == "dog_public" or optim_alg == "uni_dog" or optim_alg == "accele_dog" or optim_alg == "unixgrad" or optim_alg == "accelegrad"')
dog_df = generate_subopt(dog_df)
dog_df = generate_dog_data(dog_df)

In [ ]:
dog_df['dataset'].unique()

In [ ]:
filtered_df = dog_df.query('dataset == "wds/imagenet"')

In [ ]:
filtered_df['batch_size'].unique()

In [ ]:
filtered_df = filtered_df.query('batch_size == 32 or batch_size == 512 or batch_size == 8192 or batch_size == 128')
# .query('batch_size == 4 or batch_size == 256 or batch_size == 16384')

In [ ]:
filtered_df['batch_size'].unique()

In [ ]:
batch_df = filtered_df#.query('batch_size == 4096')
make_sensitivity_plots(batch_df, max_epoch_vals=[5, 20, 100], y_lim='auto')  #[100, 400, 1000] #[1000, 10000, 50000]

In [ ]:
batch_df = filtered_df.copy()#.query('batch_size == 4096')
best = np.nanmin(batch_df['train/loss'].apply(np.nanmin))
batch_df['train/loss'] -= best
make_sensitivity_plots(batch_df, max_epoch_vals=[5, 20, 100], y_lim=[1e-1,10.], y_axis='train/loss') # [1000, 10000, 50000] # [100, 400, 1000] #'auto'

In [ ]:
batch_df = filtered_df.copy()#.query('batch_size == 4096')
best = np.nanmin(batch_df['train/loss'].apply(np.nanmin))
batch_df['train/loss'] -= best
make_sensitivity_plots(batch_df, max_epoch_vals=[1, 20, 100], y_lim='auto', y_axis='train/rbar') # [1000, 10000, 50000] # [100, 400, 1000] #'auto'

# Adam betas

In [ ]:
markers = {'unixgrad': '+', 'accele_dog': '*', 'uni_dog': 's', 'dog_public': 'o', 'accelegrad': 'p'} 
colors = {'unixgrad': 'blue', 'accele_dog': 'orange', 'uni_dog': 'purple', 'dog_public': 'red', 'accelegrad': 'tab:purple'}
colors_bounded = {'unixgrad': {"True": 'blue', "False": 'cyan'}, 'accelegrad': {"True": 'tab:purple', "False": 'tab:pink'}}
nice_names = {'unixgrad': 'UniXGrad', 'accele_dog': 'A-DoG', 'uni_dog': 'U-DoG', 'dog_public': 'DoG', 'accelegrad': 'AcceleGrad'}
names_order = ['accele_dog', 'uni_dog', 'dog_public', 'unixgrad', 'accelegrad']
marker_size=7

import matplotlib.colors as mcolors

cmap = plt.cm.viridis
def transform(val):
    return -np.log10(1 - val)
norm = mcolors.Normalize(vmin=transform(0), vmax=transform(0.9999))

def make_sensitivity_plots(big_df_filt, epoch_selection = 'last', max_epoch_vals = [100, 400, 1000, 4000, 10000, 20000], 
                           rowsplit_keys = ['lr'], x_axis = 'beta1', y_axis = 'test/constant_8/accuracy', baseline_alg_mask = f'optim_alg in ("adamw",)',
                           baseline_alg_split_keys = ['batch_size'], auto_y_lim_range = 0.05, y_lim = [0.8, 0.95],
                           proposed_alg_mask = f'optim_alg.str.contains("dog")', proposed_alg_split_keys = ['optim_alg'], fig=None, axes=None, colorbar=False, legend_ncol=5):
    pretty_mode = False
    x_axis_categorical = True
  
    nrows = 1
    ncols = len(max_epoch_vals)
    assert not ((fig is None) ^ (axes is None))
    create_legend = fig is None

    direction = 'max'
    if 'loss' in y_axis or 'subopt' in y_axis:
        direction = 'min'
    
    for rowsplit_vals, row_df in modified_groupby(big_df_filt,rowsplit_keys):
        hlines = []

        baseline_alg_df = row_df.query(baseline_alg_mask)
        proposed_alg_df = row_df.query(proposed_alg_mask)
        
        if len(baseline_alg_df) == 0:
            continue
    
        base_size = (7, 7) if not pretty_mode else (4, 4)
        if fig is None:
            fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(base_size[0] * ncols, base_size[1] * nrows), facecolor='w', sharey='row')
        if nrows * ncols == 1:
            axes = np.array(axes)
        axes = axes.reshape(nrows, ncols)
    
        for j, max_epoch in enumerate(max_epoch_vals):
            plt.sca(axes[0, j])
            axes[0, j].clear()
            
            if len(baseline_alg_df[y_axis].iloc[0].truncate(after=max_epoch)) == 0:
                continue
                
            baselines_to_plot = {}
            
            for i, (baseline_split_vals, baseline_split_df) in enumerate(baseline_alg_df.groupby(baseline_alg_split_keys, dropna=False)):
                if epoch_selection == 'best':
                    if direction == 'max':
                        baseline_split_df['selected_y'] = baseline_split_df[y_axis].apply(lambda x: x.truncate(after=max_epoch).max())
                    else:
                        baseline_split_df['selected_y'] = baseline_split_df[y_axis].apply(lambda x: x.truncate(after=max_epoch).min())
                elif epoch_selection == 'last':
                    baseline_split_df['selected_y'] = baseline_split_df[y_axis].apply(lambda x: x.truncate(after=max_epoch).iloc[-1])
                else:
                    raise ValueError('?')
                if direction == 'max':
                    to_plot = baseline_split_df.groupby(x_axis, dropna=False)['selected_y'].max()
                else:
                    to_plot = baseline_split_df.groupby(x_axis, dropna=False)['selected_y'].min()

                if x_axis_categorical:
                    # to_plot.index = to_plot.index.fillna('none')
                    to_plot.index = pd.Categorical(to_plot.index.values) #, categories=sorted(baseline_alg_df[x_axis].unique()))
                # pdb.set_trace()
            
                baselines_to_plot[config_str(baseline_alg_split_keys, baseline_split_vals)] = to_plot
                
            baselines_to_plot = pd.DataFrame(baselines_to_plot)
            baselines_to_plot.index = baselines_to_plot.index.fillna('none')
            # styles = [dict(color=get_color(i), marker=get_marker(i)) for i in range(len(baselines_to_plot))]
            # styles = [get_color(i) for i in range(len(baselines_to_plot))]
            h = baselines_to_plot.plot(ax=plt.gca(), legend=False, ms=marker_size)
            for i, ln in enumerate(h.lines):
                ln.set_marker(get_marker(i))
                # ln.set_color(get_color(i))
                #momentum = float(baselines_to_plot.columns[i].split('momentum = ')[-1])
                # pdb.set_trace()
                #ln.set_color(get_momentum_color(momentum))
                opt_alg = baselines_to_plot.columns[i].split(', ')[0].split('optim_alg = ')[-1]
                #ln.set_color(colors_bounded[opt_alg][bounded])
    
            for ii, (proposed_split_vals, proposed_split_df) in enumerate(proposed_alg_df.groupby(proposed_alg_split_keys, dropna=False)):
                if epoch_selection == 'best':
                    if direction == 'max':
                        proposed_split_df['selected_y'] = proposed_split_df[y_axis].apply(lambda x: x.truncate(after=max_epoch).max())
                    else:
                        proposed_split_df['selected_y'] = proposed_split_df[y_axis].apply(lambda x: x.truncate(after=max_epoch).min())
                elif epoch_selection == 'last':
                    proposed_split_df['selected_y'] = proposed_split_df[y_axis].apply(lambda x: x.truncate(after=max_epoch).iloc[-1])
                else:
                    raise ValueError('?')
                if direction == 'max':
                    to_plot = proposed_split_df['selected_y'].max()
                else:
                    to_plot = proposed_split_df['selected_y'].min()

                # pdb.set_trace()
                if len(proposed_split_vals) > 1 or proposed_split_vals[0] not in colors:
                    color = colors.get(baseline_split_vals, get_color(-1-ii))
                    plt.axhline(to_plot, color=color, ls='--', lw=2, label=f'{config_str(proposed_alg_split_keys, proposed_split_vals)}')
                else:
                    color = colors[proposed_split_vals[0]]
                    marker = markers[proposed_split_vals[0]]
                    hlines.append((color, marker, to_plot))
                    plt.axhline(to_plot, color=color, lw=2, label=f'{config_str(proposed_alg_split_keys, proposed_split_vals)}')

    
            plt.title(f'Max {baseline_alg_df[y_axis].iloc[0].index.name} = {max_epoch}')
            if not x_axis_categorical:
                plt.xscale('log')
            if y_lim == 'auto':
                # y_max = min(1.0, pd.concat([baseline_alg_df, proposed_alg_df], axis=0)[y_axis].apply(lambda x: x.max()).max() + auto_y_lim_range)
                # y_min = min(0.0, pd.concat([baseline_alg_df, proposed_alg_df], axis=0)[y_axis].apply(lambda x: x.min()).min() - auto_y_lim_range)
                y_max = min(1.0, np.nanpercentile(pd.concat([baseline_alg_df, proposed_alg_df], axis=0)[y_axis].apply(lambda x: x.max()), 95) + auto_y_lim_range)
                y_min = max(0.0, np.nanpercentile(pd.concat([baseline_alg_df, proposed_alg_df], axis=0)[y_axis].apply(lambda x: x.min()), 5) - auto_y_lim_range)
                #plt.ylim(y_min, y_max)
            else:    
                plt.ylim(y_lim)
            if 'loss' in y_axis or 'subopt' in y_axis or 'rbar' in y_axis:
                plt.yscale('log')
            plt.ylabel(y_axis)
            plt.xlabel(x_axis)
    
            plt.grid(True, which='major', axis='y')
            if not x_axis_categorical:
                plt.grid(True, which='major', axis='x')
            plt.grid(True, which='minor', axis='y', color=[0.9, 0.9, 0.9], linestyle='--')
    
    
        handles, labels = plt.gca().get_legend_handles_labels()
    
        if create_legend:
            fig.legend(
                    prop={"size": 14},
            #         columnspacing=0.0,
                    ncol=legend_ncol,
                    handles=handles,
                    labels=labels,
                    loc='upper center',
                    bbox_to_anchor=[0.5, -0.05],
                    frameon=True,
                    markerscale=1.0,
                )
            #dataset_name = big_df_filt.iloc[0].dataset.replace('/', '+')
            model_name = big_df_filt.iloc[0].model.replace('/', '+')
            
            fig.suptitle(f'{config_str(rowsplit_keys, rowsplit_vals)}')
        
            plt.tight_layout()
            plt.show()
            #filename = f"figs/sensitivity/{dataset_name}-{config_str(rowsplit_keys, rowsplit_vals)} {model_name} sensitivity.pdf"
            #fig.savefig(filename, dpi=300, bbox_inches='tight')
            #fig.savefig(filename.replace('pdf', 'png'), dpi=300, bbox_inches='tight')
        else:
            metric = y_axis.replace('constant_8/', '').replace('/', ' ')
            plt.gca().set_title(f'{metric.capitalize()} after {max_epoch_vals[0]} batches')
            plt.xlabel('learning rate')

            if colorbar:
                # Create a color bar with the transformed scale
                sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
                sm.set_array([])
                cbar = plt.colorbar(sm, ax=plt.gca())
                # Set the color bar labels to the original real values
                cbar.set_ticks(transformed_momentum_values)
                cbar.set_ticklabels([f'{val:.4f}' for val in momentum_values])
                cbar.set_label('Momentum')

            x_min, x_max = plt.gca().get_xlim()
            # Plot a single marker at the rightmost edge
            for color, marker, to_plot in hlines:
                plt.gca().plot(10**-5, to_plot, marker=marker, color=color, markersize=marker_size)

In [ ]:
conv_big_df = read_experiments(['24-06-01-openwebtext_adam_betas_clipped',
                                '24-05-31-openwebtext_adam_betas_clipped_more_bs',
                                '24-06-02-openwebtext_adam_betas_clipped_more_lr',
                                '24-06-02-openwebtext_adam_betas_clipped_more_lr2'
                          ], index_field='epoch', folder='results')

In [ ]:
make_sensitivity_plots(conv_big_df, max_epoch_vals=[50], y_lim=[3.8, 6.5], y_axis='test/poly_32_1/loss', epoch_selection='best')

In [ ]:
make_sensitivity_plots(conv_big_df, max_epoch_vals=[50], y_lim=[3.8, 6.5], y_axis='test/last/loss', epoch_selection='last')

In [ ]:
make_sensitivity_plots(conv_big_df, max_epoch_vals=[50], y_lim=[3.8, 6.5], y_axis='test/last/loss', epoch_selection='last', rowsplit_keys=[])

In [ ]:
make_sensitivity_plots(conv_big_df, max_epoch_vals=[50], y_lim=[3.8, 6.5], y_axis='test/last/loss', epoch_selection='best', rowsplit_keys=[])

# openwebtext_dog

In [ ]:
dog_df = read_experiments(['24-06-02-openwebtext_dog_clipped',
                          ], index_field='epoch', folder='results')

In [ ]:
dog_df = generate_dog_data(dog_df, with_effective_lr=False)

In [ ]:
make_sensitivity_plots(dog_df, max_epoch_vals=[50], y_lim=[3.8, 6.5], y_axis='test/poly_32_1/loss', epoch_selection='best',
                           rowsplit_keys = [], x_axis = 'batch_size', baseline_alg_mask = f'optim_alg.str.contains("dog")',
                           baseline_alg_split_keys = ['optim_alg', 'dog_granularity'],
                           proposed_alg_mask = f'not optim_alg.str.contains("dog")', proposed_alg_split_keys = ['optim_alg'], legend_ncol=2)

In [ ]:
make_sensitivity_plots(dog_df, max_epoch_vals=[50], y_lim=[3.8, 6.5], y_axis='test/poly_32_1/loss', epoch_selection='last',
                           rowsplit_keys = [], x_axis = 'batch_size', baseline_alg_mask = f'optim_alg.str.contains("dog")',
                           baseline_alg_split_keys = ['optim_alg', 'dog_granularity'],
                           proposed_alg_mask = f'not optim_alg.str.contains("dog")', proposed_alg_split_keys = ['optim_alg'], legend_ncol=2)

In [ ]:
by_steps = False
limit = None #7000 #

In [ ]:
#datasets_to_show = ['vtab/clevr_distance', 'vtab/svhn']
big_df_filt = dog_df#.query('dog_granularity == "all"')

row_fields = ['batch_size']
col_fields = ['wd']
legend_fields = ['optim_alg', 'dog_granularity']
specific_fields = {}#{'optim_alg': 'dog', 'loss':'log'}
y_field = 'train/rbar' #'test/av/loss'#'test/constant_8/accuracy' #   'test/last/empirical_subopt' # 'test/last/accuracy' # 'train/empirical_subopt' # 'train/rbar' # 'train/alpha'#'train/eta' # 

big_df_filt = big_df_filt.sort_values(row_fields + col_fields + legend_fields)
n_rows = big_df_filt[row_fields].value_counts().shape[0]
n_cols = big_df_filt[col_fields].value_counts().shape[0]

print (n_rows, n_cols)

base_size = (6,4)
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(base_size[0] * n_cols, base_size[1] * n_rows), facecolor='w')
if n_rows * n_cols == 1:
    axes = np.array(axes)
axes = axes.reshape(n_rows, n_cols)

for i_row, (row_vals, row_df) in enumerate(modified_groupby(big_df_filt,row_fields)):
    for i_col, (col_vals, row_col_df) in enumerate(modified_groupby(row_df,col_fields)):
        plt.sca(axes[i_row, i_col])
        for i_entry, (legend_vals, entry_df) in enumerate(modified_groupby(row_col_df,legend_fields)):
            skip = False
            for field in specific_fields:
                idx = legend_fields.index(field)
                skip = skip or not (legend_vals[idx]== specific_fields[field])
            if skip:
                continue

            assert (len(entry_df[y_field]) == 1)
            rbar = entry_df[y_field].iloc[0]
            if legend_vals[1] == 'param':
                rbar = np.linalg.norm(np.stack(rbar), axis=1)

            # we average over repeated entries with the same legend key; a better thing would be to add error bars / shaded region
            plt.plot(rbar, color=get_color(i_entry), marker=get_marker(i_entry), label=config_str(legend_fields, legend_vals))
        if i_col == 0:
            plt.ylabel(config_str(row_fields, row_vals, sep='\n'))
        if i_col == n_cols - 1:  # assuming elements in the plot are the same for every row
            plt.legend(loc='upper left', bbox_to_anchor=[1.01, 1.025])
        if i_row == n_rows - 1:
            pass#plt.xlabel(entry_df.iloc[0][y_field].index.name)
        if 'subopt' in y_field:
            plt.yscale('log')
            plt.ylim([0.00, 1.0])
        if 'accuracy' in y_field:
            
            #plt.ylim([0.5, 0.8])
            pass

        #plt.ylim([2.00, 3.0])
        plt.yscale('log')

        xlim = plt.xlim()
        if (not (limit is None)) and xlim[1] > limit:
            plt.xlim([-1, limit])
            xlim = plt.xlim()
        
        plt.grid(True, which='major', axis='y')
        plt.grid(True, which='major', axis='x')
        plt.grid(True, which='minor', axis='y', color=[0.9, 0.9, 0.9], linestyle='--')
        plt.title(config_str(col_fields, col_vals))

fig.suptitle(y_field)
#plt.tight_layout()
plt.show()  

In [ ]:
#datasets_to_show = ['vtab/clevr_distance', 'vtab/svhn']
big_df_filt = conv_big_df.query('dog_granularity == "all"')

row_fields = ['batch_size']
col_fields = ['wd']
legend_fields = ['optim_alg']
specific_fields = {}#{'optim_alg': 'dog', 'loss':'log'}
y_field = 'train/rbar' #'test/av/loss'#'test/constant_8/accuracy' #   'test/last/empirical_subopt' # 'test/last/accuracy' # 'train/empirical_subopt' # 'train/rbar' # 'train/alpha'#'train/eta' # 

big_df_filt = big_df_filt.sort_values(row_fields + col_fields + legend_fields)
n_rows = big_df_filt[row_fields].value_counts().shape[0]
n_cols = big_df_filt[col_fields].value_counts().shape[0]

print (n_rows, n_cols)

base_size = (6,4)
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(base_size[0] * n_cols, base_size[1] * n_rows), facecolor='w')
if n_rows * n_cols == 1:
    axes = np.array(axes)
axes = axes.reshape(n_rows, n_cols)

for i_row, (row_vals, row_df) in enumerate(modified_groupby(big_df_filt,row_fields)):
    for i_col, (col_vals, row_col_df) in enumerate(modified_groupby(row_df,col_fields)):
        plt.sca(axes[i_row, i_col])
        for i_entry, (legend_vals, entry_df) in enumerate(modified_groupby(row_col_df,legend_fields)):
            skip = False
            for field in specific_fields:
                idx = legend_fields.index(field)
                skip = skip or not (legend_vals[idx]== specific_fields[field])
            if skip:
                continue
            # we average over repeated entries with the same legend key; a better thing would be to add error bars / shaded region
            plt.plot(sum(entry_df[y_field]) / len(entry_df[y_field]), color=get_color(i_entry), marker=get_marker(i_entry), label=config_str(legend_fields, legend_vals))
        if i_col == 0:
            plt.ylabel(config_str(row_fields, row_vals, sep='\n'))
        if i_col == n_cols - 1:  # assuming elements in the plot are the same for every row
            plt.legend(loc='upper left', bbox_to_anchor=[1.01, 1.025])
        if i_row == n_rows - 1:
            pass#plt.xlabel(entry_df.iloc[0][y_field].index.name)
        if 'subopt' in y_field:
            plt.yscale('log')
            plt.ylim([0.00, 1.0])
        if 'accuracy' in y_field:
            
            #plt.ylim([0.5, 0.8])
            pass

        #plt.ylim([2.00, 3.0])
        #plt.yscale('log')

        xlim = plt.xlim()
        if (not (limit is None)) and xlim[1] > limit:
            plt.xlim([-1, limit])
            xlim = plt.xlim()
        
        plt.grid(True, which='major', axis='y')
        plt.grid(True, which='major', axis='x')
        plt.grid(True, which='minor', axis='y', color=[0.9, 0.9, 0.9], linestyle='--')
        plt.title(config_str(col_fields, col_vals))

fig.suptitle(y_field)
#plt.tight_layout()
plt.show()  

In [ ]:
#datasets_to_show = ['vtab/clevr_distance', 'vtab/svhn']
big_df_filt = conv_big_df.query('dog_granularity == "param" and optim_alg != "dog"')

row_fields = ['batch_size']
col_fields = ['wd']
legend_fields = ['optim_alg']
specific_fields = {}#{'optim_alg': 'dog', 'loss':'log'}
y_field = 'train/rbar' #'test/av/loss'#'test/constant_8/accuracy' #   'test/last/empirical_subopt' # 'test/last/accuracy' # 'train/empirical_subopt' # 'train/rbar' # 'train/alpha'#'train/eta' # 

big_df_filt = big_df_filt.sort_values(row_fields + col_fields + legend_fields)
n_rows = big_df_filt[row_fields].value_counts().shape[0]
n_cols = big_df_filt[col_fields].value_counts().shape[0]

print (n_rows, n_cols)

base_size = (6,4)
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(base_size[0] * n_cols, base_size[1] * n_rows), facecolor='w')
if n_rows * n_cols == 1:
    axes = np.array(axes)
axes = axes.reshape(n_rows, n_cols)

for i_row, (row_vals, row_df) in enumerate(modified_groupby(big_df_filt,row_fields)):
    for i_col, (col_vals, row_col_df) in enumerate(modified_groupby(row_df,col_fields)):
        plt.sca(axes[i_row, i_col])
        for i_entry, (legend_vals, entry_df) in enumerate(modified_groupby(row_col_df,legend_fields)):
            skip = False
            for field in specific_fields:
                idx = legend_fields.index(field)
                skip = skip or not (legend_vals[idx]== specific_fields[field])
            if skip:
                continue
            # we average over repeated entries with the same legend key; a better thing would be to add error bars / shaded region
            assert (len(entry_df[y_field]) == 1)
            rbar = entry_df[y_field].iloc[0]
            rbar = np.stack(rbar)
            # print(rbar.shape) # epoch, layer
            for l in [0,1] + list(range(10, rbar.shape[1], 20)) + [rbar.shape[1]-2, rbar.shape[1]-1]: #range(rbar.shape[1]):
                mu = 1.
                if l == 0 or l == rbar.shape[1]-1:
                    mu = 224/50432
                plt.plot(rbar[:, l]*mu, color=get_color(i_entry*rbar.shape[1] + l), marker=get_marker(i_entry*rbar.shape[1] + l), label=config_str(legend_fields, legend_vals)+"_" + str(l))
        if i_col == 0:
            plt.ylabel(config_str(row_fields, row_vals, sep='\n'))
        if i_col == n_cols - 1:  # assuming elements in the plot are the same for every row
            plt.legend(loc='upper left', bbox_to_anchor=[1.01, 1.025])
        if i_row == n_rows - 1:
            pass#plt.xlabel(entry_df.iloc[0][y_field].index.name)
        if 'subopt' in y_field:
            plt.yscale('log')
            plt.ylim([0.00, 1.0])
        if 'accuracy' in y_field:
            
            #plt.ylim([0.5, 0.8])
            pass

        #plt.ylim([2.00, 3.0])
        plt.yscale('log')

        xlim = plt.xlim()
        if (not (limit is None)) and xlim[1] > limit:
            plt.xlim([-1, limit])
            xlim = plt.xlim()
        
        plt.grid(True, which='major', axis='y')
        plt.grid(True, which='major', axis='x')
        plt.grid(True, which='minor', axis='y', color=[0.9, 0.9, 0.9], linestyle='--')
        plt.title(config_str(col_fields, col_vals))

fig.suptitle(y_field)
#plt.tight_layout()
plt.show()  

In [ ]:
#datasets_to_show = ['vtab/clevr_distance', 'vtab/svhn']
big_df_filt = conv_big_df.query('dog_granularity == "param" and optim_alg == "dog"')

row_fields = ['batch_size']
col_fields = ['wd']
legend_fields = ['optim_alg']
specific_fields = {}#{'optim_alg': 'dog', 'loss':'log'}
y_field = 'train/rbar' #'test/av/loss'#'test/constant_8/accuracy' #   'test/last/empirical_subopt' # 'test/last/accuracy' # 'train/empirical_subopt' # 'train/rbar' # 'train/alpha'#'train/eta' # 

big_df_filt = big_df_filt.sort_values(row_fields + col_fields + legend_fields)
n_rows = big_df_filt[row_fields].value_counts().shape[0]
n_cols = big_df_filt[col_fields].value_counts().shape[0]

print (n_rows, n_cols)

base_size = (6,4)
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(base_size[0] * n_cols, base_size[1] * n_rows), facecolor='w')
if n_rows * n_cols == 1:
    axes = np.array(axes)
axes = axes.reshape(n_rows, n_cols)

for i_row, (row_vals, row_df) in enumerate(modified_groupby(big_df_filt,row_fields)):
    for i_col, (col_vals, row_col_df) in enumerate(modified_groupby(row_df,col_fields)):
        plt.sca(axes[i_row, i_col])
        for i_entry, (legend_vals, entry_df) in enumerate(modified_groupby(row_col_df,legend_fields)):
            skip = False
            for field in specific_fields:
                idx = legend_fields.index(field)
                skip = skip or not (legend_vals[idx]== specific_fields[field])
            if skip:
                continue
            # we average over repeated entries with the same legend key; a better thing would be to add error bars / shaded region
            assert (len(entry_df[y_field]) == 1)
            rbar = entry_df[y_field].iloc[0]
            rbar = np.stack(rbar)
            # print(rbar.shape) # epoch, layer
            for l in [0,1] + list(range(10, rbar.shape[1], 20)) + [rbar.shape[1]-2, rbar.shape[1]-1]: #range(rbar.shape[1]):
                plt.plot(rbar[:, l], color=get_color(i_entry*rbar.shape[1] + l), marker=get_marker(i_entry*rbar.shape[1] + l), label=config_str(legend_fields, legend_vals)+"_" + str(l))
        if i_col == 0:
            plt.ylabel(config_str(row_fields, row_vals, sep='\n'))
        if i_col == n_cols - 1:  # assuming elements in the plot are the same for every row
            plt.legend(loc='upper left', bbox_to_anchor=[1.01, 1.025])
        if i_row == n_rows - 1:
            pass#plt.xlabel(entry_df.iloc[0][y_field].index.name)
        if 'subopt' in y_field:
            plt.yscale('log')
            plt.ylim([0.00, 1.0])
        if 'accuracy' in y_field:
            
            #plt.ylim([0.5, 0.8])
            pass

        #plt.ylim([2.00, 3.0])
        plt.yscale('log')

        xlim = plt.xlim()
        if (not (limit is None)) and xlim[1] > limit:
            plt.xlim([-1, limit])
            xlim = plt.xlim()
        
        plt.grid(True, which='major', axis='y')
        plt.grid(True, which='major', axis='x')
        plt.grid(True, which='minor', axis='y', color=[0.9, 0.9, 0.9], linestyle='--')
        plt.title(config_str(col_fields, col_vals))

fig.suptitle(y_field)
#plt.tight_layout()
plt.show()  

In [ ]:
#datasets_to_show = ['vtab/clevr_distance', 'vtab/svhn']
big_df_filt = dog_df#.query('granularity_reverse == True')

row_fields = ['batch_size']
col_fields = ['wd']
legend_fields = ['optim_alg', 'dog_granularity']
specific_fields = {}#{'optim_alg': 'dog', 'loss':'log'}
y_field = 'test/poly_32_1/loss' # 'train/grad_norm' #'test/av/loss'#  'test/last/empirical_subopt' # 'test/last/accuracy' # 'train/empirical_subopt' # 'train/rbar' # 'train/alpha'#'train/eta' # 

big_df_filt = big_df_filt.sort_values(row_fields + col_fields + legend_fields)
n_rows = big_df_filt[row_fields].value_counts().shape[0]
n_cols = big_df_filt[col_fields].value_counts().shape[0]

print (n_rows, n_cols)

base_size = (6,4)
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(base_size[0] * n_cols, base_size[1] * n_rows), facecolor='w')
if n_rows * n_cols == 1:
    axes = np.array(axes)
axes = axes.reshape(n_rows, n_cols)

for i_row, (row_vals, row_df) in enumerate(modified_groupby(big_df_filt,row_fields)):
    for i_col, (col_vals, row_col_df) in enumerate(modified_groupby(row_df,col_fields)):
        plt.sca(axes[i_row, i_col])
        for i_entry, (legend_vals, entry_df) in enumerate(modified_groupby(row_col_df,legend_fields)):
            skip = False
            for field in specific_fields:
                idx = legend_fields.index(field)
                skip = skip or not (legend_vals[idx]== specific_fields[field])
            if skip:
                continue
            # we average over repeated entries with the same legend key; a better thing would be to add error bars / shaded region
            assert (len(entry_df[y_field]) == 1)
            # print(rbar.shape) # epoch, layer
            plt.plot(entry_df[y_field].iloc[0], color=get_color(i_entry), marker=get_marker(i_entry), label=config_str(legend_fields, legend_vals))
        if i_col == 0:
            plt.ylabel(config_str(row_fields, row_vals, sep='\n'))
        if i_col == n_cols - 1:  # assuming elements in the plot are the same for every row
            plt.legend(loc='upper left', bbox_to_anchor=[1.01, 1.025])
        if i_row == n_rows - 1:
            pass#plt.xlabel(entry_df.iloc[0][y_field].index.name)
        if 'subopt' in y_field:
            plt.yscale('log')
            plt.ylim([0.00, 1.0])
        if 'accuracy' in y_field:
            
            #plt.ylim([0.5, 0.8])
            pass

        plt.ylim([3.8, 6.5])
        plt.yscale('log')

        xlim = plt.xlim()
        if (not (limit is None)) and xlim[1] > limit:
            plt.xlim([-1, limit])
            xlim = plt.xlim()
        
        plt.grid(True, which='major', axis='y')
        plt.grid(True, which='major', axis='x')
        plt.grid(True, which='minor', axis='y', color=[0.9, 0.9, 0.9], linestyle='--')
        plt.title(config_str(col_fields, col_vals))

fig.suptitle(y_field)
#plt.tight_layout()
plt.show()  

# openwebtext_dadapt

In [ ]:
conv_big_df = read_experiments(['24-06-03-openwebtext_dadapt',
                          ], index_field='epoch', folder='results')

In [ ]:
make_sensitivity_plots(conv_big_df, max_epoch_vals=[50], y_lim=[3.8, 6.5], y_axis='test/poly_32_1/loss', epoch_selection='best',
                           rowsplit_keys = ['wd'], x_axis = 'batch_size', baseline_alg_mask = f'not optim_alg.str.contains("dog")',
                           baseline_alg_split_keys = ['optim_alg', 'lr_scheduler'],
                           proposed_alg_mask = f'optim_alg.str.contains("dog")', proposed_alg_split_keys = ['optim_alg'], legend_ncol=2)

In [ ]:
make_sensitivity_plots(conv_big_df, max_epoch_vals=[50], y_lim=[3.8, 6.5], y_axis='test/poly_32_1/loss', epoch_selection='last',
                           rowsplit_keys = ['wd'], x_axis = 'batch_size', baseline_alg_mask = f'not optim_alg.str.contains("dog")',
                           baseline_alg_split_keys = ['optim_alg', 'lr_scheduler'],
                           proposed_alg_mask = f'optim_alg.str.contains("dog")', proposed_alg_split_keys = ['optim_alg'], legend_ncol=2)

In [ ]:
make_sensitivity_plots(conv_big_df, max_epoch_vals=[50], y_lim=[3.8, 6.5], y_axis='test/poly_32_1/loss', epoch_selection='best',
                           rowsplit_keys = [], x_axis = 'batch_size', baseline_alg_mask = f'not optim_alg.str.contains("dog")',
                           baseline_alg_split_keys = ['optim_alg', 'lr_scheduler'],
                           proposed_alg_mask = f'optim_alg.str.contains("dog")', proposed_alg_split_keys = ['optim_alg'], legend_ncol=2)

In [ ]:
make_sensitivity_plots(conv_big_df, max_epoch_vals=[50], y_lim=[3.8, 6.5], y_axis='test/poly_32_1/loss', epoch_selection='last',
                           rowsplit_keys = [], x_axis = 'batch_size', baseline_alg_mask = f'not optim_alg.str.contains("dog")',
                           baseline_alg_split_keys = ['optim_alg', 'lr_scheduler'],
                           proposed_alg_mask = f'optim_alg.str.contains("dog")', proposed_alg_split_keys = ['optim_alg'], legend_ncol=2)

In [ ]:
by_steps = False
limit = None #7000 #

In [ ]:
#datasets_to_show = ['vtab/clevr_distance', 'vtab/svhn']
big_df_filt = conv_big_df.query('wd == 0.0')

row_fields = ['batch_size']
col_fields = ['lr_scheduler']
legend_fields = ['optim_alg']
specific_fields = {}#{'optim_alg': 'dog', 'loss':'log'}
y_field = 'train/d' #'test/av/loss'#'test/constant_8/accuracy' #   'test/last/empirical_subopt' # 'test/last/accuracy' # 'train/empirical_subopt' # 'train/rbar' # 'train/alpha'#'train/eta' # 

big_df_filt = big_df_filt.sort_values(row_fields + col_fields + legend_fields)
n_rows = big_df_filt[row_fields].value_counts().shape[0]
n_cols = big_df_filt[col_fields].value_counts().shape[0]

print (n_rows, n_cols)

base_size = (6,4)
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(base_size[0] * n_cols, base_size[1] * n_rows), facecolor='w')
if n_rows * n_cols == 1:
    axes = np.array(axes)
axes = axes.reshape(n_rows, n_cols)

for i_row, (row_vals, row_df) in enumerate(modified_groupby(big_df_filt,row_fields)):
    for i_col, (col_vals, row_col_df) in enumerate(modified_groupby(row_df,col_fields)):
        plt.sca(axes[i_row, i_col])
        for i_entry, (legend_vals, entry_df) in enumerate(modified_groupby(row_col_df,legend_fields)):
            skip = False
            for field in specific_fields:
                idx = legend_fields.index(field)
                skip = skip or not (legend_vals[idx]== specific_fields[field])
            if skip:
                continue
            # we average over repeated entries with the same legend key; a better thing would be to add error bars / shaded region
            plt.plot(sum(entry_df[y_field]) / len(entry_df[y_field]), color=get_color(i_entry), marker=get_marker(i_entry), label=config_str(legend_fields, legend_vals))
        if i_col == 0:
            plt.ylabel(config_str(row_fields, row_vals, sep='\n'))
        if i_col == n_cols - 1:  # assuming elements in the plot are the same for every row
            plt.legend(loc='upper left', bbox_to_anchor=[1.01, 1.025])
        if i_row == n_rows - 1:
            pass#plt.xlabel(entry_df.iloc[0][y_field].index.name)
        if 'subopt' in y_field:
            plt.yscale('log')
            plt.ylim([0.00, 1.0])
        if 'accuracy' in y_field:
            
            #plt.ylim([0.5, 0.8])
            pass

        #plt.ylim([2.00, 3.0])
        #plt.yscale('log')

        xlim = plt.xlim()
        if (not (limit is None)) and xlim[1] > limit:
            plt.xlim([-1, limit])
            xlim = plt.xlim()
        
        plt.grid(True, which='major', axis='y')
        plt.grid(True, which='major', axis='x')
        plt.grid(True, which='minor', axis='y', color=[0.9, 0.9, 0.9], linestyle='--')
        plt.title(config_str(col_fields, col_vals))

fig.suptitle(y_field)
#plt.tight_layout()
plt.show()  

# DoG granularities

In [ ]:
dog_gran_df = read_experiments(['24-06-14-openwebtext_dog_multiple_granularities',
                          ], index_field='epoch', folder='results')

In [ ]:
dog_gran_df = generate_dog_data(dog_gran_df, with_effective_lr=False, adog=False)

In [ ]:
make_sensitivity_plots(dog_gran_df.query('granularity_reverse == False'), max_epoch_vals=[50], y_lim=[3.8, 6.5], y_axis='test/poly_32_1/loss', epoch_selection='best',
                           rowsplit_keys = [], x_axis = 'batch_size', baseline_alg_mask = f'optim_alg.str.contains("dog")',
                           baseline_alg_split_keys = ['granularity_attentions', 'granularity_feed_forward'],
                           proposed_alg_mask = f'not optim_alg.str.contains("dog")', proposed_alg_split_keys = ['optim_alg'], legend_ncol=2)

In [ ]:
make_sensitivity_plots(dog_gran_df.query('granularity_reverse == True'), max_epoch_vals=[50], y_lim=[3.8, 6.5], y_axis='test/poly_32_1/loss', epoch_selection='best',
                           rowsplit_keys = [], x_axis = 'batch_size', baseline_alg_mask = f'optim_alg.str.contains("dog")',
                           baseline_alg_split_keys = ['granularity_attentions', 'granularity_feed_forward'],
                           proposed_alg_mask = f'not optim_alg.str.contains("dog")', proposed_alg_split_keys = ['optim_alg'], legend_ncol=2)

In [ ]:
by_steps = False
limit = None #7000 #

In [ ]:
#datasets_to_show = ['vtab/clevr_distance', 'vtab/svhn']
big_df_filt = dog_gran_df#.query('granularity_reverse == True')

row_fields = ['batch_size']
col_fields = ['granularity_reverse']
legend_fields = ['granularity_attentions', 'granularity_feed_forward']
specific_fields = {}#{'optim_alg': 'dog', 'loss':'log'}
y_field = 'train/rbar' #'test/av/loss'#'test/constant_8/accuracy' #   'test/last/empirical_subopt' # 'test/last/accuracy' # 'train/empirical_subopt' # 'train/rbar' # 'train/alpha'#'train/eta' # 

big_df_filt = big_df_filt.sort_values(row_fields + col_fields + legend_fields)
n_rows = big_df_filt[row_fields].value_counts().shape[0]
n_cols = big_df_filt[col_fields].value_counts().shape[0]

print (n_rows, n_cols)

base_size = (6,4)
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(base_size[0] * n_cols, base_size[1] * n_rows), facecolor='w')
if n_rows * n_cols == 1:
    axes = np.array(axes)
axes = axes.reshape(n_rows, n_cols)

for i_row, (row_vals, row_df) in enumerate(modified_groupby(big_df_filt,row_fields)):
    for i_col, (col_vals, row_col_df) in enumerate(modified_groupby(row_df,col_fields)):
        plt.sca(axes[i_row, i_col])
        for i_entry, (legend_vals, entry_df) in enumerate(modified_groupby(row_col_df,legend_fields)):
            skip = False
            for field in specific_fields:
                idx = legend_fields.index(field)
                skip = skip or not (legend_vals[idx]== specific_fields[field])
            if skip:
                continue
            # we average over repeated entries with the same legend key; a better thing would be to add error bars / shaded region
            assert (len(entry_df[y_field]) == 1)
            rbar = entry_df[y_field].iloc[0]
            rbar = np.linalg.norm(np.stack(rbar), axis=1)
            # print(rbar.shape) # epoch, layer
            plt.plot(rbar, color=get_color(i_entry), marker=get_marker(i_entry), label=config_str(legend_fields, legend_vals))
        if i_col == 0:
            plt.ylabel(config_str(row_fields, row_vals, sep='\n'))
        if i_col == n_cols - 1:  # assuming elements in the plot are the same for every row
            plt.legend(loc='upper left', bbox_to_anchor=[1.01, 1.025])
        if i_row == n_rows - 1:
            pass#plt.xlabel(entry_df.iloc[0][y_field].index.name)
        if 'subopt' in y_field:
            plt.yscale('log')
            plt.ylim([0.00, 1.0])
        if 'accuracy' in y_field:
            
            #plt.ylim([0.5, 0.8])
            pass

        #plt.ylim([2.00, 3.0])
        plt.yscale('log')

        xlim = plt.xlim()
        if (not (limit is None)) and xlim[1] > limit:
            plt.xlim([-1, limit])
            xlim = plt.xlim()
        
        plt.grid(True, which='major', axis='y')
        plt.grid(True, which='major', axis='x')
        plt.grid(True, which='minor', axis='y', color=[0.9, 0.9, 0.9], linestyle='--')
        plt.title(config_str(col_fields, col_vals))

fig.suptitle(y_field)
#plt.tight_layout()
plt.show()  

In [ ]:
#datasets_to_show = ['vtab/clevr_distance', 'vtab/svhn']
big_df_filt = dog_gran_df#.query('granularity_reverse == True')

row_fields = ['batch_size']
col_fields = ['granularity_reverse']
legend_fields = ['granularity_attentions', 'granularity_feed_forward']
specific_fields = {}#{'optim_alg': 'dog', 'loss':'log'}
y_field = 'train/grad_norm' #'test/av/loss'#'test/constant_8/accuracy' #   'test/last/empirical_subopt' # 'test/last/accuracy' # 'train/empirical_subopt' # 'train/rbar' # 'train/alpha'#'train/eta' # 

big_df_filt = big_df_filt.sort_values(row_fields + col_fields + legend_fields)
n_rows = big_df_filt[row_fields].value_counts().shape[0]
n_cols = big_df_filt[col_fields].value_counts().shape[0]

print (n_rows, n_cols)

base_size = (6,4)
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(base_size[0] * n_cols, base_size[1] * n_rows), facecolor='w')
if n_rows * n_cols == 1:
    axes = np.array(axes)
axes = axes.reshape(n_rows, n_cols)

for i_row, (row_vals, row_df) in enumerate(modified_groupby(big_df_filt,row_fields)):
    for i_col, (col_vals, row_col_df) in enumerate(modified_groupby(row_df,col_fields)):
        plt.sca(axes[i_row, i_col])
        for i_entry, (legend_vals, entry_df) in enumerate(modified_groupby(row_col_df,legend_fields)):
            skip = False
            for field in specific_fields:
                idx = legend_fields.index(field)
                skip = skip or not (legend_vals[idx]== specific_fields[field])
            if skip:
                continue
            # we average over repeated entries with the same legend key; a better thing would be to add error bars / shaded region
            assert (len(entry_df[y_field]) == 1)
            grad_norm = entry_df[y_field].iloc[0]
            # print(rbar.shape) # epoch, layer
            plt.plot(grad_norm, color=get_color(i_entry), marker=get_marker(i_entry), label=config_str(legend_fields, legend_vals))
        if i_col == 0:
            plt.ylabel(config_str(row_fields, row_vals, sep='\n'))
        if i_col == n_cols - 1:  # assuming elements in the plot are the same for every row
            plt.legend(loc='upper left', bbox_to_anchor=[1.01, 1.025])
        if i_row == n_rows - 1:
            pass#plt.xlabel(entry_df.iloc[0][y_field].index.name)
        if 'subopt' in y_field:
            plt.yscale('log')
            plt.ylim([0.00, 1.0])
        if 'accuracy' in y_field:
            
            #plt.ylim([0.5, 0.8])
            pass

        #plt.ylim([2.00, 3.0])
        plt.yscale('log')

        xlim = plt.xlim()
        if (not (limit is None)) and xlim[1] > limit:
            plt.xlim([-1, limit])
            xlim = plt.xlim()
        
        plt.grid(True, which='major', axis='y')
        plt.grid(True, which='major', axis='x')
        plt.grid(True, which='minor', axis='y', color=[0.9, 0.9, 0.9], linestyle='--')
        plt.title(config_str(col_fields, col_vals))

fig.suptitle(y_field)
#plt.tight_layout()
plt.show()  

In [ ]:
#datasets_to_show = ['vtab/clevr_distance', 'vtab/svhn']
big_df_filt = dog_gran_df#.query('granularity_reverse == True')

row_fields = ['batch_size']
col_fields = ['granularity_reverse']
legend_fields = ['granularity_attentions', 'granularity_feed_forward']
specific_fields = {}#{'optim_alg': 'dog', 'loss':'log'}
y_field = 'test/poly_32_1/loss' # 'train/grad_norm' #'test/av/loss'#  'test/last/empirical_subopt' # 'test/last/accuracy' # 'train/empirical_subopt' # 'train/rbar' # 'train/alpha'#'train/eta' # 

big_df_filt = big_df_filt.sort_values(row_fields + col_fields + legend_fields)
n_rows = big_df_filt[row_fields].value_counts().shape[0]
n_cols = big_df_filt[col_fields].value_counts().shape[0]

print (n_rows, n_cols)

base_size = (6,4)
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(base_size[0] * n_cols, base_size[1] * n_rows), facecolor='w')
if n_rows * n_cols == 1:
    axes = np.array(axes)
axes = axes.reshape(n_rows, n_cols)

for i_row, (row_vals, row_df) in enumerate(modified_groupby(big_df_filt,row_fields)):
    for i_col, (col_vals, row_col_df) in enumerate(modified_groupby(row_df,col_fields)):
        plt.sca(axes[i_row, i_col])
        for i_entry, (legend_vals, entry_df) in enumerate(modified_groupby(row_col_df,legend_fields)):
            skip = False
            for field in specific_fields:
                idx = legend_fields.index(field)
                skip = skip or not (legend_vals[idx]== specific_fields[field])
            if skip:
                continue
            # we average over repeated entries with the same legend key; a better thing would be to add error bars / shaded region
            assert (len(entry_df[y_field]) == 1)
            # print(rbar.shape) # epoch, layer
            plt.plot(entry_df[y_field].iloc[0], color=get_color(i_entry), marker=get_marker(i_entry), label=config_str(legend_fields, legend_vals))
        if i_col == 0:
            plt.ylabel(config_str(row_fields, row_vals, sep='\n'))
        if i_col == n_cols - 1:  # assuming elements in the plot are the same for every row
            plt.legend(loc='upper left', bbox_to_anchor=[1.01, 1.025])
        if i_row == n_rows - 1:
            pass#plt.xlabel(entry_df.iloc[0][y_field].index.name)
        if 'subopt' in y_field:
            plt.yscale('log')
            plt.ylim([0.00, 1.0])
        if 'accuracy' in y_field:
            
            #plt.ylim([0.5, 0.8])
            pass

        plt.ylim([3.8, 6.5])
        plt.yscale('log')

        xlim = plt.xlim()
        if (not (limit is None)) and xlim[1] > limit:
            plt.xlim([-1, limit])
            xlim = plt.xlim()
        
        plt.grid(True, which='major', axis='y')
        plt.grid(True, which='major', axis='x')
        plt.grid(True, which='minor', axis='y', color=[0.9, 0.9, 0.9], linestyle='--')
        plt.title(config_str(col_fields, col_vals))

fig.suptitle(y_field)
#plt.tight_layout()
plt.show()  

# A-DoG granularities

In [ ]:
adog_gran_df = read_experiments(['24-06-17-openwebtext_a-dog_multiple_granularities',
                          ], index_field='epoch', folder='results')

In [ ]:
adog_gran_df = generate_dog_data(adog_gran_df, with_effective_lr=False, dog=False)

In [ ]:
make_sensitivity_plots(adog_gran_df.query('granularity_reverse == False'), max_epoch_vals=[50], y_lim=[3.8, 5], y_axis='test/poly_32_1/loss', epoch_selection='best',
                           rowsplit_keys = [], x_axis = 'batch_size', baseline_alg_mask = f'optim_alg.str.contains("dog")',
                           baseline_alg_split_keys = ['granularity_attentions', 'granularity_feed_forward'],
                           proposed_alg_mask = f'not optim_alg.str.contains("dog")', proposed_alg_split_keys = ['optim_alg'], legend_ncol=2)

In [ ]:
make_sensitivity_plots(adog_gran_df.query('granularity_reverse == True'), max_epoch_vals=[50], y_lim=[3.8, 5], y_axis='test/poly_32_1/loss', epoch_selection='best',
                           rowsplit_keys = [], x_axis = 'batch_size', baseline_alg_mask = f'optim_alg.str.contains("dog")',
                           baseline_alg_split_keys = ['granularity_attentions', 'granularity_feed_forward'],
                           proposed_alg_mask = f'not optim_alg.str.contains("dog")', proposed_alg_split_keys = ['optim_alg'], legend_ncol=2)

In [ ]:
by_steps = False
limit = None #7000 #

In [ ]:
#datasets_to_show = ['vtab/clevr_distance', 'vtab/svhn']
big_df_filt = adog_gran_df#.query('granularity_reverse == True')

row_fields = ['batch_size']
col_fields = ['granularity_reverse']
legend_fields = ['granularity_attentions', 'granularity_feed_forward']
specific_fields = {}#{'optim_alg': 'dog', 'loss':'log'}
y_field = 'train/rbar' #'test/av/loss'#'test/constant_8/accuracy' #   'test/last/empirical_subopt' # 'test/last/accuracy' # 'train/empirical_subopt' # 'train/rbar' # 'train/alpha'#'train/eta' # 

big_df_filt = big_df_filt.sort_values(row_fields + col_fields + legend_fields)
n_rows = big_df_filt[row_fields].value_counts().shape[0]
n_cols = big_df_filt[col_fields].value_counts().shape[0]

print (n_rows, n_cols)

base_size = (6,4)
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(base_size[0] * n_cols, base_size[1] * n_rows), facecolor='w')
if n_rows * n_cols == 1:
    axes = np.array(axes)
axes = axes.reshape(n_rows, n_cols)

for i_row, (row_vals, row_df) in enumerate(modified_groupby(big_df_filt,row_fields)):
    for i_col, (col_vals, row_col_df) in enumerate(modified_groupby(row_df,col_fields)):
        plt.sca(axes[i_row, i_col])
        for i_entry, (legend_vals, entry_df) in enumerate(modified_groupby(row_col_df,legend_fields)):
            skip = False
            for field in specific_fields:
                idx = legend_fields.index(field)
                skip = skip or not (legend_vals[idx]== specific_fields[field])
            if skip:
                continue
            # we average over repeated entries with the same legend key; a better thing would be to add error bars / shaded region
            assert (len(entry_df[y_field]) == 1)
            rbar = entry_df[y_field].iloc[0]
            rbar = np.linalg.norm(np.stack(rbar), axis=1)
            # print(rbar.shape) # epoch, layer
            plt.plot(rbar, color=get_color(i_entry), marker=get_marker(i_entry), label=config_str(legend_fields, legend_vals))
        if i_col == 0:
            plt.ylabel(config_str(row_fields, row_vals, sep='\n'))
        if i_col == n_cols - 1:  # assuming elements in the plot are the same for every row
            plt.legend(loc='upper left', bbox_to_anchor=[1.01, 1.025])
        if i_row == n_rows - 1:
            pass#plt.xlabel(entry_df.iloc[0][y_field].index.name)
        if 'subopt' in y_field:
            plt.yscale('log')
            plt.ylim([0.00, 1.0])
        if 'accuracy' in y_field:
            
            #plt.ylim([0.5, 0.8])
            pass

        #plt.ylim([2.00, 3.0])
        plt.yscale('log')

        xlim = plt.xlim()
        if (not (limit is None)) and xlim[1] > limit:
            plt.xlim([-1, limit])
            xlim = plt.xlim()
        
        plt.grid(True, which='major', axis='y')
        plt.grid(True, which='major', axis='x')
        plt.grid(True, which='minor', axis='y', color=[0.9, 0.9, 0.9], linestyle='--')
        plt.title(config_str(col_fields, col_vals))

fig.suptitle(y_field)
#plt.tight_layout()
plt.show()  

In [ ]:
#datasets_to_show = ['vtab/clevr_distance', 'vtab/svhn']
big_df_filt = adog_gran_df#.query('granularity_reverse == True')

row_fields = ['batch_size']
col_fields = ['granularity_reverse']
legend_fields = ['granularity_attentions', 'granularity_feed_forward']
specific_fields = {}#{'optim_alg': 'dog', 'loss':'log'}
y_field = 'test/poly_8_1/loss' # 'train/grad_norm' #'test/av/loss'#  'test/last/empirical_subopt' # 'test/last/accuracy' # 'train/empirical_subopt' # 'train/rbar' # 'train/alpha'#'train/eta' # 

big_df_filt = big_df_filt.sort_values(row_fields + col_fields + legend_fields)
n_rows = big_df_filt[row_fields].value_counts().shape[0]
n_cols = big_df_filt[col_fields].value_counts().shape[0]

print (n_rows, n_cols)

base_size = (6,4)
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(base_size[0] * n_cols, base_size[1] * n_rows), facecolor='w')
if n_rows * n_cols == 1:
    axes = np.array(axes)
axes = axes.reshape(n_rows, n_cols)

for i_row, (row_vals, row_df) in enumerate(modified_groupby(big_df_filt,row_fields)):
    for i_col, (col_vals, row_col_df) in enumerate(modified_groupby(row_df,col_fields)):
        plt.sca(axes[i_row, i_col])
        for i_entry, (legend_vals, entry_df) in enumerate(modified_groupby(row_col_df,legend_fields)):
            skip = False
            for field in specific_fields:
                idx = legend_fields.index(field)
                skip = skip or not (legend_vals[idx]== specific_fields[field])
            if skip:
                continue
            # we average over repeated entries with the same legend key; a better thing would be to add error bars / shaded region
            assert (len(entry_df[y_field]) == 1)
            # print(rbar.shape) # epoch, layer
            plt.plot(entry_df[y_field].iloc[0], color=get_color(i_entry), marker=get_marker(i_entry), label=config_str(legend_fields, legend_vals))
        if i_col == 0:
            plt.ylabel(config_str(row_fields, row_vals, sep='\n'))
        if i_col == n_cols - 1:  # assuming elements in the plot are the same for every row
            plt.legend(loc='upper left', bbox_to_anchor=[1.01, 1.025])
        if i_row == n_rows - 1:
            pass#plt.xlabel(entry_df.iloc[0][y_field].index.name)
        if 'subopt' in y_field:
            plt.yscale('log')
            plt.ylim([0.00, 1.0])
        if 'accuracy' in y_field:
            
            #plt.ylim([0.5, 0.8])
            pass

        plt.ylim([3.8, 6.5])
        plt.yscale('log')

        xlim = plt.xlim()
        if (not (limit is None)) and xlim[1] > limit:
            plt.xlim([-1, limit])
            xlim = plt.xlim()
        
        plt.grid(True, which='major', axis='y')
        plt.grid(True, which='major', axis='x')
        plt.grid(True, which='minor', axis='y', color=[0.9, 0.9, 0.9], linestyle='--')
        plt.title(config_str(col_fields, col_vals))

fig.suptitle(y_field)
#plt.tight_layout()
plt.show()  

# Temp

In [ ]:
adog_gran_df = read_experiments(['24-06-20-openwebtext_a-dog_momentum_granularity_no_output_embed-norm',
                          ], index_field='epoch', folder='results')

In [ ]:
adog_gran_df = generate_dog_data(adog_gran_df, with_effective_lr=False, dog=False)

In [ ]:
make_sensitivity_plots(adog_gran_df, max_epoch_vals=[50], y_lim=[3.8, 5], y_axis='test/poly_32_1/loss', epoch_selection='best',
                           rowsplit_keys = [], x_axis = 'batch_size', baseline_alg_mask = f'optim_alg.str.contains("dog")',
                           baseline_alg_split_keys = ['momentum'],
                           proposed_alg_mask = f'not optim_alg.str.contains("dog")', proposed_alg_split_keys = ['optim_alg'], legend_ncol=2)

# DoG multi

In [ ]:
a_dog_df = read_experiments(['24-06-02-openwebtext_dog_clipped',
                            '24-06-14-openwebtext_dog_multiple_granularities',
                            '24-06-17-openwebtext_a-dog_multiple_granularities',
                            '24-06-01-openwebtext_adam_betas_clipped',
                                '24-05-31-openwebtext_adam_betas_clipped_more_bs',
                                '24-06-02-openwebtext_adam_betas_clipped_more_lr',
                                '24-06-02-openwebtext_adam_betas_clipped_more_lr2'
                          ], index_field='epoch', folder='results')

In [ ]:
a_dog_df = generate_subopt(a_dog_df)

In [ ]:
df = a_dog_df.query('(optim_alg=="dog" and dog_granularity == "param") or (optim_alg=="adamw" and beta1==0.9)')
make_sensitivity_plots(df, max_epoch_vals=[50], y_lim=[3.8, 5], y_axis='test/poly_32_1/loss', epoch_selection='best',
                           rowsplit_keys = [], x_axis = 'batch_size', baseline_alg_mask = f'optim_alg.str.contains("dog") or optim_alg.str.contains("adam")',
                           baseline_alg_split_keys = ['optim_alg','granularity_attentions', 'granularity_feed_forward', 'granularity_reverse'],
                           proposed_alg_mask = f'not optim_alg.str.contains("dog") and not optim_alg.str.contains("adam")', proposed_alg_split_keys = ['optim_alg'], legend_ncol=2)

In [ ]:
df = a_dog_df.query('(optim_alg=="accele_dog" and dog_granularity == "param") or (optim_alg=="adamw")')
make_sensitivity_plots(df, max_epoch_vals=[50], y_lim=[3.8, 5], y_axis='test/poly_32_1/loss', epoch_selection='best',
                           rowsplit_keys = [], x_axis = 'batch_size', baseline_alg_mask = f'optim_alg.str.contains("dog") or optim_alg.str.contains("adam")',
                           baseline_alg_split_keys = ['optim_alg','granularity_attentions', 'granularity_feed_forward', 'granularity_reverse'],
                           proposed_alg_mask = f'not optim_alg.str.contains("dog") and not optim_alg.str.contains("adam")', proposed_alg_split_keys = ['optim_alg'], legend_ncol=2)

In [ ]:
df = a_dog_df.query('optim_alg=="accele_dog" and dog_granularity == "param"')
make_sensitivity_plots(df, max_epoch_vals=[50], y_lim=[3.8, 5], y_axis='test/poly_32_1/loss', epoch_selection='best',
                           rowsplit_keys = [], x_axis = 'batch_size', baseline_alg_mask = f'optim_alg.str.contains("dog")',
                           baseline_alg_split_keys = ['granularity_attentions', 'granularity_feed_forward', 'granularity_reverse'],
                           proposed_alg_mask = f'not optim_alg.str.contains("dog")', proposed_alg_split_keys = ['optim_alg'], legend_ncol=2)

# openhermes dog

In [ ]:
conv_big_df = read_experiments(['24-05-09-openhermes_dog_6layers_decay_to_init'
                          ], index_field='epoch', folder='results')

In [ ]:
conv_big_df = generate_subopt(conv_big_df)

In [ ]:
conv_big_df = select_av(conv_big_df, 'poly_32_1')

In [ ]:
by_steps = False
limit = None #7000 #

In [ ]:
#datasets_to_show = ['vtab/clevr_distance', 'vtab/svhn']
big_df_filt = conv_big_df.query('dog_granularity == "param"')

row_fields = ['wd']
col_fields = ['batch_size']
legend_fields = ['optim_alg']
specific_fields = {}#{'optim_alg': 'dog', 'loss':'log'}
y_field = 'test/av/loss'#'test/constant_8/accuracy' #  'train/rbar' # 'test/last/empirical_subopt' # 'test/last/accuracy' # 'train/empirical_subopt' # 'train/rbar' # 'train/alpha'#'train/eta' # 

big_df_filt = big_df_filt.sort_values(row_fields + col_fields + legend_fields)
n_rows = big_df_filt[row_fields].value_counts().shape[0]
n_cols = big_df_filt[col_fields].value_counts().shape[0]

print (n_rows, n_cols)

base_size = (6,4)
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(base_size[0] * n_cols, base_size[1] * n_rows), facecolor='w')
if n_rows * n_cols == 1:
    axes = np.array(axes)
axes = axes.reshape(n_rows, n_cols)

for i_row, (row_vals, row_df) in enumerate(modified_groupby(big_df_filt,row_fields)):
    for i_col, (col_vals, row_col_df) in enumerate(modified_groupby(row_df,col_fields)):
        plt.sca(axes[i_row, i_col])
        for i_entry, (legend_vals, entry_df) in enumerate(modified_groupby(row_col_df,legend_fields)):
            skip = False
            for field in specific_fields:
                idx = legend_fields.index(field)
                skip = skip or not (legend_vals[idx]== specific_fields[field])
            if skip:
                continue
            # we average over repeated entries with the same legend key; a better thing would be to add error bars / shaded region
            plt.plot(sum(entry_df[y_field]) / len(entry_df[y_field]), color=get_color(i_entry), marker=get_marker(i_entry), label=config_str(legend_fields, legend_vals))
        if i_col == 0:
            plt.ylabel(config_str(row_fields, row_vals, sep='\n'))
        if i_col == n_cols - 1:  # assuming elements in the plot are the same for every row
            plt.legend(loc='upper left', bbox_to_anchor=[1.01, 1.025])
        if i_row == n_rows - 1:
            plt.xlabel(entry_df.iloc[0][y_field].index.name)
        if 'subopt' in y_field:
            plt.yscale('log')
            plt.ylim([0.00, 1.0])
        if 'accuracy' in y_field:
            
            #plt.ylim([0.5, 0.8])
            pass

        plt.ylim([2.00, 3.0])
        #plt.yscale('log')

        xlim = plt.xlim()
        if (not (limit is None)) and xlim[1] > limit:
            plt.xlim([-1, limit])
            xlim = plt.xlim()
        
        plt.grid(True, which='major', axis='y')
        plt.grid(True, which='major', axis='x')
        plt.grid(True, which='minor', axis='y', color=[0.9, 0.9, 0.9], linestyle='--')
        plt.title(config_str(col_fields, col_vals))

fig.suptitle(y_field)
#plt.tight_layout()
plt.show()  

In [ ]:
get_train_loss_tables(conv_big_df, rows=['optim_alg', 'dog_granularity', 'batch_size'])['layers=6_hidden-dim=224']

In [ ]:
get_test_loss_tables(conv_big_df, rows=['optim_alg', 'dog_granularity', 'batch_size'])['layers=6_hidden-dim=224']

# openhermes adam

In [ ]:
conv_adam = read_experiments(['24-05-08-openhermes_adam_betas'
                          ], index_field='epoch', folder='results')

In [ ]:
conv_adam = generate_subopt(conv_adam)

In [ ]:
conv_adam = select_av(conv_adam, 'poly_32_1')

In [ ]:
by_steps = False
limit = None #7000 #

In [ ]:
#datasets_to_show = ['vtab/clevr_distance', 'vtab/svhn']
big_df_filt = conv_adam.query('batch_size == 4')

row_fields = ['lr']
col_fields = ['wd']
legend_fields = ['beta1']
specific_fields = {}#{'optim_alg': 'dog', 'loss':'log'}
y_field = 'test/last/loss'#'test/constant_8/accuracy' #  'train/rbar' # 'test/last/empirical_subopt' # 'test/last/accuracy' # 'train/empirical_subopt' # 'train/rbar' # 'train/alpha'#'train/eta' # 

big_df_filt = big_df_filt.sort_values(row_fields + col_fields + legend_fields)
n_rows = big_df_filt[row_fields].value_counts().shape[0]
n_cols = big_df_filt[col_fields].value_counts().shape[0]

print (n_rows, n_cols)

base_size = (6,4)
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(base_size[0] * n_cols, base_size[1] * n_rows), facecolor='w')
if n_rows * n_cols == 1:
    axes = np.array(axes)
axes = axes.reshape(n_rows, n_cols)

for i_row, (row_vals, row_df) in enumerate(modified_groupby(big_df_filt,row_fields)):
    for i_col, (col_vals, row_col_df) in enumerate(modified_groupby(row_df,col_fields)):
        plt.sca(axes[i_row, i_col])
        for i_entry, (legend_vals, entry_df) in enumerate(modified_groupby(row_col_df,legend_fields)):
            skip = False
            for field in specific_fields:
                idx = legend_fields.index(field)
                skip = skip or not (legend_vals[idx]== specific_fields[field])
            if skip:
                continue
            # we average over repeated entries with the same legend key; a better thing would be to add error bars / shaded region
            plt.plot(sum(entry_df[y_field]) / len(entry_df[y_field]), color=get_color(i_entry), marker=get_marker(i_entry), label=config_str(legend_fields, legend_vals))
        if i_col == 0:
            plt.ylabel(config_str(row_fields, row_vals, sep='\n'))
        if i_col == n_cols - 1:  # assuming elements in the plot are the same for every row
            plt.legend(loc='upper left', bbox_to_anchor=[1.01, 1.025])
        if i_row == n_rows - 1:
            plt.xlabel(entry_df.iloc[0][y_field].index.name)
        if 'subopt' in y_field:
            plt.yscale('log')
            plt.ylim([0.00, 1.0])
        if 'accuracy' in y_field:
            
            #plt.ylim([0.5, 0.8])
            pass

        plt.ylim([2.40, 3.0])
        #plt.yscale('log')

        xlim = plt.xlim()
        if (not (limit is None)) and xlim[1] > limit:
            plt.xlim([-1, limit])
            xlim = plt.xlim()
        
        plt.grid(True, which='major', axis='y')
        plt.grid(True, which='major', axis='x')
        plt.grid(True, which='minor', axis='y', color=[0.9, 0.9, 0.9], linestyle='--')
        plt.title(config_str(col_fields, col_vals))

fig.suptitle(y_field)
#plt.tight_layout()
plt.show()  

In [ ]:
get_train_loss_tables(conv_adam.query('batch_size == 4'), rows=['lr', 'beta1'])['layers=6_hidden-dim=224']

In [ ]:
get_test_loss_tables(conv_adam, rows=['lr', 'beta1', 'batch_size'])['layers=6_hidden-dim=224']